# Objetivos de la práctica

En esta actividad propone desarrollar un proyecto de Ciencia de Datos respecto a el uso de bicicletas por parte de una empresa.

In [ ]:
import pandas as pd
import sqlite3
import os
print(os.getcwd())
os.chdir('./../')
print(os.getcwd())
import matplotlib.pyplot as plt

# Carga y análisis de datos

Lo primero es comprender el problema desde la fuente de datos original


## Descarga de uso de bicis

https://valencia.opendatasoft.com/pages/home/

https://valencia.opendatasoft.com/explore/dataset/valenbisi-disponibilitat-valenbisi-dsiponibilidad/table/?flg=es-es


In [ ]:
dfb = pd.read_csv('./data/raw/usobici_raw.csv')
dfb.tail()

In [ ]:
dfb.shape

In [ ]:
dfb.describe(include='all')

In [ ]:
# Tipos de datos en pandas
dfb.dtypes

In [ ]:
# Conversión de la columna fecha
dfb["updated"] = pd.to_datetime(dfb.updated)
# Rango temporal de la muestra
print(dfb["updated"].min(), dfb["updated"].max())

In [ ]:
dfb["address"].unique().shape[0]

## Datos de estaciones

In [ ]:
dfe = pd.read_csv('./data/interim/estaciones.csv')
dfe.tail()

## Descarga de AEMET

https://opendata.aemet.es/centrodedescargas/inicio

https://opendata.aemet.es/opendata/api/observacion/convencional/todas

aemet_metadata.json


In [ ]:
dfm = pd.read_csv('./data/raw/aemet.csv')
dfm.tail()

In [ ]:
dfm.shape

In [ ]:
dfm.describe(include='all')

In [ ]:
dfm.dtypes

In [ ]:
# Conversión de la columna updated
dfm["fecha"] = pd.to_datetime(dfm.fecha)
# Rango temporal de la muestra
print(dfm["fecha"].min(), dfm["fecha"].max())

In [ ]:
dfm["ubi"].unique().shape[0]

In [ ]:
# Búsqueda de datos faltantes
sert = pd.date_range(start=dfm['fecha'].min(), 
                     end=dfm['fecha'].max(), freq='h')
data = {
    'datetime': sert,
    'value': range(len(sert))
}
df = pd.DataFrame(data)
full_range_df = pd.DataFrame(sert, columns=['fecha'])
merged_df = full_range_df.merge(dfm, on='fecha', how='left')
missing_data = merged_df[merged_df['tamin'].isna()]
print("Registros faltantes:")
print(missing_data)

# Integración de datos

Los datos tienen una muestra que es una selección de estaciones del barrio de Ruzafa.

In [ ]:
dfb = pd.read_csv('./data/interim/usoestameteo.csv')
dfb.tail()

In [ ]:
# Estructura del DF
dfb.shape, dfb.columns

In [ ]:
dfb.head()

In [ ]:
# Conversión de la columna updated
dfb["fecha"] = pd.to_datetime(dfb.fecha)
# Rango temporal de la muestra
print(dfb["fecha"].min(), dfb["fecha"].max())

In [ ]:
dfb.sort_values("fecha", ascending=True, inplace=True)
dfb.tail()

In [ ]:
# Visualizar datos nulos por columnas
dfb.isna().sum()

In [ ]:
# Identificar las filas que tienen valores nulos 
# y conocer su distribución en la serie revisar variables: 
# count_out, address
dfb.loc[dfb['nombre'].isna(), ['fecha', 'count_out', 'nombre']]

# Eliminar valores nulos

In [ ]:
# No lo hacemos persistente el cambio, 
# solo muestra el df total resultado tras eliminar
dfb.dropna(how='any')['nombre'].value_counts().sort_index()

In [ ]:
# en caso de haber nulos eliminaría los registros
df2 = dfb.dropna(how='any').copy()
df2.shape

In [ ]:
df2.isna().sum()

# Análisis exploratorio

In [ ]:
df2 = pd.read_csv('./data/interim/usobarriosmeteo.csv')
df2.tail()

In [ ]:
var_analisis = "name"

In [ ]:
# Número de estaciones disponibles
# Estaciones únicas
df2[var_analisis].unique()
# Número de estaciones disponibles
# df2[var_analisis].unique().shape[0]

In [ ]:
# Recuento de registros por estación
df2[var_analisis].value_counts()
# Vemos si hay variaciones en toda la muestra
df2[var_analisis].value_counts().describe()

In [ ]:
# Conversión de la columna updated
df2["fecha"] = pd.to_datetime(df2.fecha)
# Rango temporal de la muestra
print(df2["fecha"].min(), df2["fecha"].max())

In [ ]:
# Descripción de todas las variables disponibles
df2.describe(include="all")

In [ ]:
# Muestra por barrios el uso de bicicleta medio y desviación
df2.groupby(by=var_analisis).agg(av_mean=pd.NamedAgg(column="uso_bici", aggfunc="mean"),
                            av_std=pd.NamedAgg(column="uso_bici", aggfunc="std"))\
                            .sort_values("av_std", ascending=False)

In [ ]:
# Pivot table es útil para hacer agrupaciones con distintas variables categóricas
df2.pivot_table(index='name', columns=df2.fecha.dt.hour, values='uso_bici')

In [ ]:
# Estilos para visualización de tablas
headers = {
    'selector': 'th.col_heading',
    'props': 'background-color: #5E17EB; color: white;'
}

index_style = {
    'selector': 'th.index_name',
    'props': 'background-color: #5E17EB; color: white;'
}

In [ ]:
df2.pivot_table(index=df2.fecha.dt.hour, columns='name', values='uso_bici').round(2)\
.style.set_table_styles([headers, index_style])\
.background_gradient(cmap='viridis')

In [ ]:
# Combinación de tablas con visualizaciones para resaltar tendencias
df2.pivot_table(index=df2.fecha.dt.hour, columns='name', values='uso_bici').round(2).abs()\
    .style.set_table_styles([headers, index_style])\
    .set_properties(**{'background-color': 'green', 'color': 'green'})\
    .bar(color='#BEEAE5')

## Estudio por barrio

In [ ]:
# Filtrado de datos usando .loc
df2.loc[df2['name']=='RUSSAFA', :]

In [ ]:
df2.loc[df2['name']=='RUSSAFA', :]\
    .pivot_table(index=df2.fecha.dt.hour, columns='name', values='uso_bici').round(2).abs()\
    .style.set_table_styles([headers, index_style])\
    .set_properties(**{'background-color': 'green', 'color': 'green'})\
    .bar(color='#BEEAE5')

In [ ]:
# Filtra para comparar algunos de los barrios 
lista_filtro = ["RUSSAFA", "PATRAIX"]
df2.loc[df2['name'].isin(lista_filtro), :]\
    .pivot_table(index=df2.fecha.dt.hour, columns='name', values=['uso_bici', 'tamax'], aggfunc='mean')\
    .round(2).abs()\
    .style.set_table_styles([headers, index_style])\
    .set_properties(**{'background-color': 'green', 'color': 'green'})\
    .bar(color='#BEEAE5')

# Mejorar procesos de análisis

Importante ver la actividad de github:

https://github.com/ydataai/ydata-profiling
```
pip install ydata-profiling
```
[Documentación profiling](https://docs.profiling.ydata.ai/latest/)
```
pip install dtale
```

Alternativas:
- https://github.com/fbdesignpro/sweetviz
- https://github.com/AutoViML/AutoViz

In [ ]:
# Se pueden ver los datos desde el cuaderno, como muestras...
df2.sample()

In [ ]:
import dtale
from ydata_profiling import ProfileReport

In [ ]:
profile = ProfileReport(df2, title="Profiling Report")

profile.to_file("./documentos/informe_bicis.html")

## Dtale

Crear el fichero en csv:
```
df2.to_csv('./data/usobici.csv', index=False)
```
Error en versiones de python 3.12

En terminal:
```
df = pd.read_csv('./data/usobici.csv')
df["fecha"] = pd.to_datetime(df.fecha)
d = dtale.show(df)
d.open_browser()
```

# Conclusiones
- Se ha analizado el dataset